### Purpose of this notebook
The purpose of this notebook is to download WLASL dataset and upload to S3 bucket after processing as Numpy

The videos will be labeled by the 'gloss' name from the Json.
Some videos only have the required sign between Frame-A and Frame-B. These will be cropped accordingly

In [1]:
#Install missing packages
#!pip install boto3
#!pip install pyarrow
#!pip install fastparquet
#!pip install s3fs
#!pip install mediapipe
#!pip install kaggle

In [2]:
# Import required libraries
import boto3 #Video files get read through this
import cv2
import os
import pandas as pd
import math
import numpy as np
import random
import io
import s3fs # Parquet files get read through this
import zlib # For compression
import time # To calculate download time
import configparser
import requests
import psutil # Checks memory usage
import tempfile
import json
#import mediapipe as mp


In [3]:
# Let's read in the credentials file
config = configparser.ConfigParser()
config.read('credentials')

['credentials']

In [4]:
aws_access_key_id = config["default"]['aws_access_key_id']
aws_secret_access_key = config["default"]['aws_secret_access_key']
bucket_name = 'asl-capstone'
prefix = '/WLASL/'
save_path = '/content/temp_folder'
s3_URI = 's3://asl-capstone/'

In [5]:
# Create an s3 object
s3 = boto3.client('s3',aws_access_key_id = aws_access_key_id, aws_secret_access_key = aws_secret_access_key,
                  region_name = 'us-west-2')

In [6]:
# Create an S3FS object
fs = s3fs.S3FileSystem(key=aws_access_key_id, secret=aws_secret_access_key) # Define an S3FS object

In [7]:
# Let's read the WLASL json file to map the number to the word
filename = "wlasl-dataset/WLASL_v0.3.json"
wlasl_df = pd.read_json(filename)

In [8]:
display(wlasl_df.head())
print(wlasl_df.shape)
#print(wlasl_df['instances'][0])

,gloss,instances
0,book,"[{'bbox': [385, 37, 885, 720], 'fps': 25, 'fra..."
1,drink,"[{'bbox': [551, 68, 1350, 1080], 'fps': 25, 'f..."
2,computer,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_..."
3,before,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_..."
4,chair,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_..."


(2000, 2)


In [9]:
# Not all videos from the JSON exist. Let's create a function to check which ones exist
def video_features(json_data):
    video_list = []
    frame_list = []
    gloss = json_data['gloss']
    json_list = json_data['instances']
    for id in json_list:
        vid = id['video_id']
        frame_start = id['frame_start']
        frame_end = id['frame_end']
        bbox = id['bbox']
        if os.path.exists(f"wlasl-dataset/videos/{vid}.mp4"):
            features = {'word':gloss,'id':vid,'start':frame_start, 'end':frame_end,'box':bbox}
            #yield features
            video_list.append(features)
    return video_list

In [10]:
# Load the data and get the features
with open("wlasl-dataset/WLASL_v0.3.json",'r') as file:
    json_data = file.read()
json_data = json.loads(json_data)

In [11]:
# Let's extract all relevant features from the JSON
video_list = []
for data in json_data:
    video_list.append(video_features(data))

In [12]:
# Print a subset to see what the final list looks like
print(video_list[1])

[{'word': 'drink', 'id': '69302', 'start': 1, 'end': -1, 'box': [551, 68, 1350, 1080]}, {'word': 'drink', 'id': '65539', 'start': 1, 'end': -1, 'box': [153, 11, 488, 370]}, {'word': 'drink', 'id': '17710', 'start': 1, 'end': -1, 'box': [196, 15, 521, 414]}, {'word': 'drink', 'id': '17733', 'start': 1, 'end': -1, 'box': [186, 63, 551, 400]}, {'word': 'drink', 'id': '65540', 'start': 1, 'end': -1, 'box': [167, 19, 480, 370]}, {'word': 'drink', 'id': '17734', 'start': 1, 'end': -1, 'box': [183, 55, 552, 400]}, {'word': 'drink', 'id': '17711', 'start': 1, 'end': -1, 'box': [289, 9, 783, 480]}, {'word': 'drink', 'id': '17712', 'start': 1, 'end': -1, 'box': [673, 90, 1571, 1072]}, {'word': 'drink', 'id': '17713', 'start': 1, 'end': -1, 'box': [28, 0, 542, 480]}, {'word': 'drink', 'id': '17709', 'start': 1, 'end': -1, 'box': [41, 9, 257, 240]}, {'word': 'drink', 'id': '17720', 'start': 1, 'end': -1, 'box': [49, 3, 220, 192]}, {'word': 'drink', 'id': '17721', 'start': 1, 'end': -1, 'box': [66,

In [13]:
# Create a folder ~/.kaggle/ in order to use the Kaggle API
#os.makedirs(os.path.expanduser('~/.kaggle/'), exist_ok=True)
#os.rename('kaggle.json', os.path.expanduser('~/.kaggle/kaggle.json')) # Move the file using os.rename
#!chmod 600 ~/.kaggle/kaggle.json  # Change permissions

In [14]:
# Download the WLASL dataset (Zip file) from Kaggle
#!kaggle datasets download -d risangbaskoro/wlasl-processed

In [15]:
# Unzip the downloaded file
#!unzip wlasl-processed.zip -d wlasl-dataset

In [16]:
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    if x > y:
        start_x = (x-y)/2
        end_x = start_x + y
        start_x = int(start_x)
        end_x = int(end_x)
        return frame[:, int(start_x):int(end_x)]
    else:
        return frame

In [17]:
def load_video(vid_id, max_frames=0, resize=(256,256)):
    local_video_path = "wlasl-dataset/videos/"+vid_id+".mp4"
    video_capture = cv2.VideoCapture(local_video_path)
    frames = []
    try:
      while video_capture.isOpened():
        ret, frame = video_capture.read()
        if not ret:
          break
        frame = crop_center_square(frame)
        frame = cv2.resize(frame, resize)
        frame = frame[:, :, [2, 1, 0]]
        frames.append(frame)

        if len(frames) == max_frames:
          break
    finally:
      video_capture.release()
    return np.array(frames) / 255.0

In [18]:
def save_frame_file(video_array, caption, start_frame, end_frame, counter):
    s3_uri = "s3://asl-capstone/wlasl/"
    if end_frame!=-1:
        video_array = video_array[int(start_frame):int(end_frame)]
    filename = caption+"_"+str(counter)+".npy"
    with fs.open(f"{s3_uri+filename}","wb") as f:
        np.save(f,video_array)
    caption_name = caption+".npy"
    caption_array = np.array(caption)
    with fs.open(f"{s3_uri+caption_name}","wb") as f:
        np.save(f,caption_array)
    

In [19]:
for videos in video_list[:1]:
    counter = 0
    print(videos[0])

{'word': 'book', 'id': '69241', 'start': 1, 'end': -1, 'box': [385, 37, 885, 720]}


In [20]:
print(len(video_list))

2000


In [ ]:
try:
    num_words = 813
    for videos in video_list[813:890]:
        counter = 0
        for ind_vid in videos:
            video_array = load_video(ind_vid['id'])
            save_frame_file(video_array, ind_vid['word'], ind_vid['start'], ind_vid['end'], counter)
            counter+=1
        num_words+=1
        print(f"{num_words} completed")
except:
    print("pass")

814 completed
815 completed
816 completed
817 completed
818 completed
819 completed
820 completed
821 completed
822 completed
823 completed
824 completed
825 completed
826 completed
827 completed
828 completed
829 completed
830 completed
831 completed
832 completed
833 completed
834 completed
835 completed
836 completed
837 completed
838 completed
839 completed
840 completed
841 completed
842 completed
843 completed
844 completed
845 completed
846 completed
847 completed
848 completed
849 completed
850 completed
851 completed
852 completed
853 completed
854 completed


In [ ]:
def get_instance_id():
    """Get current instance ID from metadata"""
    url = "http://169.254.169.254/latest/meta-data/instance-id"
    response = requests.get(url)
    return response.text

In [ ]:
def stop_instance(instance_id, region_name='us-west-2'):
    """Stop the EC2 instance"""
    ec2 = boto3.client('ec2', aws_access_key_id = aws_access_key_id, aws_secret_access_key = aws_secret_access_key, region_name=region_name)
    ec2.stop_instances(InstanceIds=[instance_id])

In [ ]:
# Get the current instance ID
instance_id = get_instance_id()
print(instance_id)
# Stop the instance
#stop_instance(instance_id)